#I. DATA MUNGING


##1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

- strutured dataset with foe missing data
- headers are there for this dataset
- no. of columns= ,(+1)
- format issues are there (age and shipment_id is not in number format eg. ten)
- Uniformity issues (Artist, artist)
- Number of columns are more or less than the expected
- Identification of data type


##2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)

In [0]:
#We have to define Spark Session to enter into Spark application
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appName("we47_Bread_n_Butter2_logistics").getOrCreate()

In [0]:
#reading the raw data from the source
df1=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source1",header=True,inferSchema=True)
display(df1.limit(10))
df1.printSchema()

**analysis**: initial stage of exploration shows that all the columns of the dataset is of string type

###1. Apply inferSchema and toDF to create a DF and analyse the actual data.

In [0]:
raw_df1=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source1",header=True,inferSchema=True).toDF("id","fname","lname","age","role")
display(raw_df1)
raw_df1.printSchema()

###2. Analyse the schema, datatypes, columns etc.,

In [0]:
raw_df1.printSchema()
print(raw_df1.columns)
print(raw_df1.schema)
print(raw_df1.dtypes)

###3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
print("actual count of the data:",raw_df1.count())
print(raw_df1.summary())

In [0]:
#dropping duplicates
#print("de-duplicated record (all columns) count", raw_df1.distinct().count())
#print("de-duplicated record (all columns) count",raw_df1.dropDuplicates().count())
#dropping duplicates based on specific columns
#print("de-duplicated record (id) count",raw_df1.dropDuplicates(["id"]).count())
display(raw_df1.describe())
#display(raw_df1.summary())

##A. Passive Data Munging - (File: logistics_source1 and logistics_source2)

In [0]:
#reading the 2nd source file named logistics_source2
raw_df2=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source2",header=True,inferSchema=True).toDF("id","fname","lname","age","role","hub_loc","vehicle_type")
display(raw_df2.limit(10))
raw_df2.printSchema()

In [0]:
raw_df2.printSchema()
print(raw_df2.columns)
print(raw_df2.schema)
print(raw_df2.dtypes)

In [0]:
print("actual count of the data:",raw_df2.count())
print(raw_df2.summary())

In [0]:
#dropping duplicates
#print("de-duplicated record (all columns) count", raw_df2.distinct().count())
#print("de-duplicated record (all columns) count",raw_df2.dropDuplicates().count())
#dropping duplicates based on specific columns
#print("de-duplicated record (id) count",raw_df2.dropDuplicates(["id"]).count())
display(raw_df2.describe())
#display(raw_df2.summary())

##DOUBT:
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
- shipment_id is non-numeric
- age is not an integer<br>

Count rows having: <br>
- fewer columns than expected 
- more columns than expected

In [0]:
master_v1=raw_df1
master_v2=raw_df2

##1.Shipment IDs that appear in both master_v1 and master_v2

In [0]:
common_ship_id=master_v1.select("id").distinct().intersect(master_v2.select("id").distinct())
display(common_ship_id)


##2.shipment_id is non-numeric

In [0]:
non_numeric_ship_id1=master_v1.where("id not rlike '^[0-9]+$'")
display(non_numeric_ship_id1)
non_numeric_ship_id2=master_v2.where("id not rlike '^[0-9]+$'")
display(non_numeric_ship_id2)

##3.age is not an integer

In [0]:
age_not_numeric1=master_v1.where("age not rlike '^[0-9]+$'")
display(age_not_numeric1)
age_not_numeric2=master_v2.where("age not rlike '^[0-9]+$'")
display(age_not_numeric2)


##B. Active Data Munging File: logistics_source1 and logistics_source2

##1. Combining Data + Schema Merging (Structuring)

###sub-task-1:Read both files without enforcing schema

In [0]:
raw_df1=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source1",header=True,inferSchema=True)
display(raw_df1)
raw_df1.printSchema()
raw_df2=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source2",header=True,inferSchema=True)
display(raw_df2)
raw_df2.printSchema()

###sub-task-2:Align them into a single canonical schema: shipment_id, first_name, last_name, age, role, hub_location, vehicle_type, data_source

In [0]:
#strt1="shipment_id int,first_name string,last_name string,age int,role string"
raw_df1=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source1",header=True)
display(raw_df1.limit(10))
raw_df1.printSchema()

#strt2="shipment_id int,first_name string,last_name string,age int,role string,hub_location string,vehicle_type string"
raw_df2=spark.read.csv("/Volumes/workspace/default/datalake_we47logistics/logistics_source2",header=True)
display(raw_df2.limit(10))
raw_df2.printSchema()

###sub-task-3:Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
from pyspark.sql.functions import lit

In [0]:
enrich_df1=raw_df1.withColumn("data_source",lit("system1"))
display(enrich_df1)
enrich_df1.printSchema()
enrich_df2=raw_df2.withColumn("data_source",lit("system2"))
display(enrich_df2)
enrich_df2.printSchema()

##note: merging both the dataframes

In [0]:
enrich_df1.write.orc("/Volumes/workspace/default/datalake_we47logistics/logistics_target",mode="overwrite")
enrich_df2.write.orc("/Volumes/workspace/default/datalake_we47logistics/logistics_target",mode="append")
merged_df=spark.read.orc("/Volumes/workspace/default/datalake_we47logistics/logistics_target",mergeSchema=True)
display(merged_df)
merged_df.printSchema()

##2. Cleansing, Scrubbing:

In [0]:
print("actual count of the data:",merged_df.count())
print(merged_df.summary())

###sub-task-1:Cleansing (removal of unwanted datasets)

####Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role

In [0]:
cleansed_df1=merged_df.na.drop(how="any",subset=["shipment_id","role"])
display(cleansed_df1)
print("actual count of the data:",cleansed_df1.count())

####Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name

In [0]:
cleansed_df2=cleansed_df1.na.drop(how="all",subset=["first_name","last_name"])
display(cleansed_df2)
print("actual count of the data:",cleansed_df2.count())

####Join Readiness Rule - Drop records where the join key is null: shipment_id

In [0]:
cleansed_df3=cleansed_df2.na.drop(how="all",subset=["shipment_id"])
display(cleansed_df3.limit(10))
print("actual count of the data:",cleansed_df3.count())

In [0]:
from pyspark.sql.functions import col
rejected_join_df=cleansed_df2.filter(col("shipment_id").isNull())
join_ready_df = cleansed_df2.filter(col("shipment_id").isNotNull())

print("Records before join readiness check:", cleansed_df2.count())
print("Records rejected (shipment_id is NULL):", rejected_join_df.count())
print("Records ready for join:", join_ready_df.count())

display(rejected_join_df)
display(join_ready_df)

###sub-task-2:Scrubbing (convert raw to tidy)

####Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN

In [0]:
scrubbed_df1=cleansed_df3.na.fill("UNKNOWN",subset=["vehicle_type"])
display(scrubbed_df1.limit(30))
print("actual count of the data:",scrubbed_df1.count())

####Invalid Age Replacement - Replace the following values in age: "ten" to -1 "" to -1

In [0]:
display(scrubbed_df1.filter(scrubbed_df1.age.cast('string') == 'ten'))
display(scrubbed_df1.filter(scrubbed_df1.age == ""))

In [0]:
from pyspark.sql.functions import when, col
scrubbed_df2 = scrubbed_df1.withColumn(
    "age",
    when((col("age").cast("string") == "ten") | (col("age").cast("string") == ""), -1).otherwise(col("age"))
)
display(scrubbed_df2)
print("actual count of the data:", scrubbed_df2.count())

####Age Defaulting Rule - Fill NULL values in the age column with: -1

In [0]:
from pyspark.sql.functions import col

scrubbed_df3 = scrubbed_df2 .withColumn(
    "age", col("age").cast("int")
).na.fill(-1, subset=["age"])
display(scrubbed_df3)
print("actual count of the data:",scrubbed_df3.count())

####Vehicle Type Normalization - Replace inconsistent vehicle types: truck to LMV bike to TwoWheeler

In [0]:
dict1={"Truck":"LMV","Bike":"TwoWheeler"}
scrubbed_df4=scrubbed_df3.na.replace(dict1,subset=["vehicle_type"])
display(scrubbed_df4)
print("actual count of the data:",scrubbed_df4.count())

##3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

###Creating shipments Details data Dataframe creation

In [0]:
from pyspark.sql.functions import col

In [0]:
df_json = spark.read \
    .format("json") \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .option("multiLine", "true") \
    .load("/Volumes/workspace/default/datalake_we47logistics/logistics_shipment_detail_3000.json")

In [0]:
display(df_json)
print(df_json.count())
print(df_json.columns)

###STANDARDIZATION STEPS:

####sub-task-1:Add a column

In [0]:
from pyspark.sql.functions import lit, initcap, current_date
standard_df1 = df_json.withColumn("domain", lit("Logistics")).withColumn("ingestion_timestamp", current_date()).withColumn("is_expedited", lit("False"))
display(standard_df1.limit(20))

####sub-task-2:Column Uniformity: role - Convert to lowercase

In [0]:
from pyspark.sql.functions import upper,lower
standard_df21=scrubbed_df4.withColumn("role",lower(col("role")))
display(standard_df21.limit(20))

In [0]:
standard_df22=standard_df1.withColumn("vehicle_type",upper(col("vehicle_type")))
display(standard_df22.limit(20))

In [0]:
standard_df23=standard_df21.withColumn("hub_location",initcap(col("hub_location")))
display(standard_df23.limit(20))

####sub-task-3:Format Standardization

In [0]:
from pyspark.sql.functions import to_date, col, round
from pyspark.sql.types import DecimalType
standard_df31=standard_df22.withColumn("shipment_date",to_date("shipment_date","yy-MM-dd"))
display(standard_df31.limit(20))
standard_df32=standard_df31.withColumn("shipment_cost",col("shipment_cost").cast(DecimalType(10,2)))
display(standard_df32.limit(20))

In [0]:
from pyspark.sql.functions import*
shipment_id_standardization={"one":"1","two":"2","three":"3","four":"4","five":"5","six":"6","seven":"7","eight":"8","nine":"9","ten":"10"}
standard_df33=standard_df23.na.replace(shipment_id_standardization,subset=["shipment_id"])
display(standard_df33)

####sub-task-4:Data Type Standardization

**Standardizing column data types to fix schema drift and enable mathematical operations.**

In [0]:
standard_df33.printSchema()

In [0]:
standard_df41=standard_df33.withColumn("age",col("age").cast("int"))
standard_df41.printSchema()

In [0]:
standard_df42=standard_df32.withColumn("shipment_weight_kg",col("shipment_weight_kg").cast("Double"))
standard_df42.printSchema()

In [0]:
standard_df43=standard_df42.withColumn("is_expedited",col("is_expedited").cast("Boolean"))
standard_df43.printSchema()

####sub-task-5:Naming Standardization

In [0]:
from pyspark.sql.functions import when, col

standard_df5 = (
    standard_df41
    .withColumnRenamed("first_name", "staff_first_name")
    .withColumnRenamed("last_name", "staff_last_name")
    .withColumnRenamed("hub_location", "origin_hub_city")
    .withColumn(
        "age",
        when(col("age").cast("string").rlike("^[0-9]+$"), col("age").cast("int")).otherwise(-1)
    )
)

display(standard_df5)

####sub-task-6:Reordering columns logically in a better standard format:

Source File: DF of Data from all 3 files
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
print(standard_df5.columns)
print(standard_df43.columns)

In [0]:
standardized_df=standard_df5.select("shipment_id","staff_first_name","staff_last_name","role","origin_hub_city").join(standard_df43.select("shipment_id","shipment_cost","ingestion_timestamp"),on="shipment_id",how="inner")
display(standardized_df)

##4.Deduplication:

###sub-task-1:Apply Record Level De-Duplication

In [0]:
dedup_df1=standardized_df.distinct()
dedup_df1.count()
display(dedup_df1)

In [0]:
print("actual count of the df",standard_df5.count())
dedup_df2=standard_df5.distinct()
print("count of the dedup df",dedup_df2.count())
display(dedup_df2.limit(20))

In [0]:
print("actual count of the df",standard_df32.count())
dedup_df3=standard_df32.distinct()
print("count of the dedup df",dedup_df3.count())
display(dedup_df3.limit(20))

###sub-task-2:Apply Column Level De-Duplication (Primary Key Enforcement)

In [0]:
dedup_df4=standardized_df.dropDuplicates(["shipment_id"])
display(dedup_df4)
dedup_df4.count()

In [0]:
dedup_df5=dedup_df2.dropDuplicates(["shipment_id"])
display(dedup_df5.limit(10))
dedup_df5.count()

In [0]:
dedup_df6=dedup_df3.dropDuplicates(["shipment_id"])
display(dedup_df6.limit(10))
dedup_df6.count()

#II.Data Enrichment - Detailing of data

##Adding of Columns (Data Enrichment)

1. Add Audit Timestamp (load_dt) 
- Source File: DF of logistics_source1 and logistics_source2
- Scenario: We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
- Action: Add a column load_dt using the function current_timestamp().

In [0]:
joined_df=dedup_df4
logistic_df=dedup_df5
json_df=dedup_df6

In [0]:
enrich_df1_logistics=logistic_df.withColumn("load_dt",current_timestamp())
display(enrich_df1_logistics.limit(10))

2. Create Full Name (full_name) 
- Source File: DF of logistics_source1 and logistics_source2
- Scenario: The reporting dashboard requires a single field for the driver's name instead of separate columns.
- Action: Create full_name by concatenating first_name and last_name with a space separator.
- Result: "Rajesh" + " " + "Kumar" -> "Rajesh Kumar"

In [0]:
enrich_df2=enrich_df1_logistics.withColumn("fullname",concat(col("staff_first_name"),lit(" "),col("staff_last_name")))
enrich_df2=enrich_df2.select("shipment_id","staff_first_name","staff_last_name","fullname","age","role","origin_hub_city","vehicle_type","data_source","load_dt")
display(enrich_df2.limit(10))


3. Define Route Segment (route_segment) 
- Source File: DF of logistics_shipment_detail_3000.json
- Scenario: The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
- Action: Combine source_city and destination_city with a hyphen.
- Result: "Chennai" + "-" + "Pune" -> "Chennai-Pune"

In [0]:
enrich_df3=json_df.withColumn("transport_lanes",concat(col("source_city"),lit("-"),col("destination_city")))
enrich_df3=enrich_df3.select("shipment_id","cargo_type","order_id","payment_mode","shipment_cost","shipment_date","shipment_status","shipment_weight_kg","vehicle_type","domain","ingestion_timestamp","is_expedited","source_city","destination_city","transport_lanes")
display(enrich_df3.limit(10))

4. Generate Vehicle Identifier (vehicle_identifier) 
- Source File: DF of logistics_shipment_detail_3000.json
- Scenario: We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
- Action: Combine vehicle_type and shipment_id to create a composite key.
- Result: "Truck" + "_" + "500001" -> "Truck_500001"

In [0]:
enrich_df4=enrich_df3.withColumn("vehicle_identifier",concat(col("vehicle_type"),lit("_"),col("shipment_id")))
display(enrich_df4.limit(10))

##Deriving of Columns (Time Intelligence)

Source File: logistics_shipment_detail_3000.json

In [0]:
last_json_df=enrich_df4

1. Derive Shipment Year (shipment_year)
- Scenario: Management needs an annual performance report to compare growth year-over-year.
- Action: Extract the year component from shipment_date.
- Result: "2024-04-23" -> 2024

In [0]:
enrich_df5=last_json_df.select("*",substring("shipment_date",1,4).alias("shipment_year"))
display(enrich_df5.limit(10))

2. Derive Shipment Month (shipment_month)
- Scenario: Analysts want to identify seasonal peaks (e.g., increased volume in December).
- Action: Extract the month component from shipment_date.
- Result: "2024-04-23" -> 4 (April)

In [0]:
enrich_df6=enrich_df5.select("*",substring("shipment_date",2,2).alias("shipment_month"))
display(enrich_df6.limit(10))

3. Flag Weekend Operations (is_weekend)
- Scenario: The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
- Action: Flag as 'True' if the shipment_date falls on a Saturday or Sunday.

In [0]:
from pyspark.sql.functions import col, to_date

enrich_df7 = enrich_df6.withColumn(
    "shipment_date",
    to_date(col("shipment_date"))
)
enrich_df7.printSchema()

In [0]:
from pyspark.sql.functions import dayofweek, when

enrich_df8 = enrich_df7.withColumn(
    "is_weekend",
    when(dayofweek(col("shipment_date")).isin(1, 7), True)
    .otherwise(False)
)

display(enrich_df8.limit(10))

4. Flag shipment status (is_expedited)
- Scenario: The Operations team needs to track shipments is IN_TRANSIT or DELIVERED.
- Action: Flag as 'True' if the shipment_status IN_TRANSIT or DELIVERED.

In [0]:
from pyspark.sql.functions import col, when

enrich_df9 =enrich_df8.withColumn(
    "is_expedited",
    when(col("shipment_status").isin("IN_TRANSIT", "DELIVERED"), True)
    .otherwise(False)
)
display(enrich_df9.limit(10))

##Enrichment/Business Logics (Calculated Fields)

Source File: logistics_shipment_detail_3000.json

In [0]:
last_json_df1=enrich_df9

1. Calculate Unit Cost (cost_per_kg)

- Scenario: The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
- Action: Divide shipment_cost by shipment_weight_kg.
- Logic: shipment_cost / shipment_weight_kg

In [0]:
from pyspark.sql.functions import col, when

enrich_df10 = last_json_df1.withColumn(
    "cost_per_kg",
    when(
        col("shipment_weight_kg").isNull() | (col("shipment_weight_kg") == 0),
        None
    ).otherwise(col("shipment_cost") / col("shipment_weight_kg"))
)
display(enrich_df10.limit(10))

2. Track Shipment Age (days_since_shipment)

- Scenario: The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
- Action: Calculate the difference in days between the current_date and the shipment_date.
- Logic: datediff(current_date(), shipment_date)

In [0]:
from pyspark.sql.functions import datediff, current_date
enrich_df11=enrich_df10.withColumn("days_since_shipment", datediff(current_date(), col("shipment_date")))
display(enrich_df11.limit(10))

3. Compute Tax Liability (tax_amount)

- Scenario: For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
- Action: Calculate 18% GST on the total shipment_cost.
- Logic: shipment_cost * 0.18

In [0]:
enrich_df12=enrich_df11.withColumn("tax_amount", col("shipment_cost") * 0.18)
display(enrich_df12.limit(10))

##Remove/Eliminate (drop, select, selectExpr)

Source File: DF of logistics_source1 and logistics_source2

In [0]:
last_updated_logistics_df=enrich_df2

1. Remove Redundant Name Columns
- Scenario: Since we have already created the full_name column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
- Action: Drop the first_name and last_name columns.
- Logic: df.drop("first_name", "last_name")

In [0]:
enrich_df13=last_updated_logistics_df.drop("staff_first_name","staff_last_name").select("shipment_id","fullname","age","role","origin_hub_city","vehicle_type","data_source","load_dt")
display(enrich_df13.limit(10))


##Splitting & Merging/Melting of Columns

Source File: DF of logistics_shipment_detail_3000.json

In [0]:
enriched_json=enrich_df12
display(enriched_json.limit(5))

###sub-task-1. Splitting (Extraction)

- Split Order Code:
- Action: Split order_id ("ORD100000") into two new columns:
- > order_prefix ("ORD")
- > order_sequence ("100000")

In [0]:
from pyspark.sql.functions import col, regexp_extract

split_df1 = enriched_json.withColumn(
    "order_prefix",
    regexp_extract(col("order_id"), "^[A-Za-z]+", 0)
).withColumn(
    "order_sequence",
    regexp_extract(col("order_id"), "[0-9]+$", 0)
)

display(split_df1.limit(5))

- Split Date:
- Action: Split shipment_date into three separate columns for partitioning:
- > ship_year (2024)
- > ship_month (4)
- > ship_day (23)

In [0]:
split_df2=split_df1.withColumn("ship_year",substring("shipment_date",1,4)).withColumn("ship_month",substring("shipment_date",6,2)).withColumn("ship_date",substring("shipment_date",9,2))
display(split_df2.limit(5))

###sub-task-2. Merging (Concatenation) 

- Create Route ID:
- Action: Merge source_city ("Chennai") and destination_city ("Pune") to create a descriptive route 
- key: route_lane ("Chennai->Pune")

In [0]:
from pyspark.sql.functions import col, concat_ws

concat_df = split_df2.withColumn(
    "route_lane",
    concat_ws("->", col("source_city"), col("destination_city"))
)
display(concat_df.limit(5))

#III.Data Customization & Processing - Application of Tailored Business Specific Rules

UDF1: Complex Incentive Calculation<BR>
Scenario: The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

- Action: Create a Python function calculate_bonus(role, age) and register it as a Spark UDF.
- Logic:

IF Role == 'Driver' AND Age > 50:<br>
Bonus = 15% of Salary (Reward for Seniority)<br>
IF Role == 'Driver' AND Age < 30:<br>
Bonus = 5% of Salary (Encouragement for Juniors)<br>
ELSE:<br>
Bonus = 0<br>
Result: A new derived column projected_bonus is generated for every row in the dataset.

In [0]:
logis_df=enrich_df13
display(logis_df)

In [0]:
#1. Import udf library
from pyspark.sql.functions import udf
#2. Create a Python function - Can only work in single local computer
def calculate_bonus(role,age):
   if role == "driver" and age > 50:
       return "15%"
   elif role == "driver" and age < 30:
       return "5%"
   else:
       return "0%"
#3. Create a UDF function
bonus_udf = udf(calculate_bonus)
#4. Apply the UDF function to the dataframe
bonus_df = logis_df.withColumn("projected_bonus",bonus_udf(col("role"),col("age")))
display(bonus_df)


UDF2: PII Masking (Privacy Compliance)
Scenario: For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

Business Rule: Show the first 2 letters, mask the middle characters with ****, and show the last letter.

Action: Create a UDF mask_identity(name).

Example:

Input: "Rajesh"
Output: "Ra****h"
Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.

In [0]:
#1. Import udf library
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#2. Create a Python function - Can only work in single local computer
def mask_identity(name):
    if name is None or len(name) < 3:
        return name
    return name[:2] + "****" + name[-1]
#3. Create a UDF function
mask_udf=udf(mask_identity)
#4. Apply the UDF function to the dataframe
mask_identity_df=bonus_df.withColumn("masked_name",mask_udf(col("fullname")))
display(mask_identity_df)